In [6]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from models.managers.db_manager import connect_to_bdd
from src.data.utils.constants import id_to_party_dict

conn = connect_to_bdd()

# Requête pour election_2022_t1 et sauvegarde dans df_election
query_election = """
    SELECT winner, code_postal, "Libellé de la commune"
    FROM election_2022_t1
"""
df_election = pd.read_sql(query_election, conn)

# Requête pour economie et sauvegarde dans df_economy
query_economy = """
    SELECT *
    FROM economie
"""
df_economy = pd.read_sql(query_economy, conn)

# Fermeture de la connexion
conn.close()

# Mapper les noms des partis politiques aux identifiants numériques
party_to_id_dict = {v: k for k, v in id_to_party_dict.items()}
df_election["winner"] = df_election["winner"].map(party_to_id_dict)

# Convertir la colonne "winner" en float
df_election["winner"] = df_election["winner"].astype(float)

# Fusionner les données électorales avec les données économiques
df_combined = pd.merge(df_election, df_economy, on="code_postal")

# Encodage des variables catégorielles
label_encoder = LabelEncoder()

# Séparation des features et de la target
X = df_combined.drop(columns=["Libellé de la commune", "winner"])
y = df_combined["winner"]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Entraînement du modèle avec 1000 itérations
model = LogisticRegression(max_iter=1000)
model.fit(X_train, y_train)

# Prédiction sur l'ensemble de test
y_pred = model.predict(X_test)

# Calcul de l'accuracy du modèle
accuracy = accuracy_score(y_test, y_pred)
print("Précision du modèle sur l'ensemble de test:", accuracy)

# Filtre sur les données de test pour conserver uniquement les données de l'Yonne
X_test_yonne = X_test[X_test["code_postal"] == 89]
y_test_yonne = y_test[X_test["code_postal"] == 89]

# Prédiction sur les données de test pour l'Yonne
y_pred_yonne = model.predict(X_test_yonne)

# Calcul de l'accuracy du modèle sur les données de l'Yonne
accuracy_yonne = accuracy_score(y_test_yonne, y_pred_yonne)
print("Précision du modèle sur les données de l'Yonne:", accuracy_yonne)

/var/folders/9z/lnzs0dyx0k1g2nvs92hf0syr0000gn/T/ipykernel_2445/1320304311.py:17: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_election = pd.read_sql(query_election, conn)
/var/folders/9z/lnzs0dyx0k1g2nvs92hf0syr0000gn/T/ipykernel_2445/1320304311.py:24: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_economy = pd.read_sql(query_economy, conn)


Précision du modèle sur l'ensemble de test: 0.5742331288343558
Précision du modèle sur les données de l'Yonne: 0.835820895522388


/Users/erwanbuisson/Library/Mobile Documents/com~apple~CloudDocs/Documents/Interpreteur Python/lib/python3.12/site-packages/sklearn/linear_model/_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
